In [ ]:
from data_utils import *

In [ ]:
import json
import os
import pickle
from pathlib import Path
from PIL import Image
import sys

## Sample an experimental configuration

Uses the `data_utils.SyntheticEC` class to randomly sample the semantic features and blindspots for a single EC.

In [ ]:
ec = SyntheticEC()
ec.sample(verbose = True)

In [ ]:
ec_dir = './data/sample_ec'

ec.save_dataset(directory = ec_dir,
               num_train_images_per_bucket = 20,
               num_val_images_per_bucket = 20,
               num_test_images_per_bucket = 20)

## Model Training: Demo

Fine-tunes a pretrained ResNet-18 using the sample EC.

In [ ]:
from torchvision.models import resnet18
import torch
from torch_utils import *

In [ ]:
# Load the model
model = resnet18(pretrained = True)

# Change the classification layer
model.fc = torch.nn.Linear(in_features = 512, out_features = 1)

In [ ]:
files, labels = load_phase(ec_dir, 'train')
dataset = ImageDataset(files, labels)
train_loader = get_loader(dataset, batch_size = 16)

# change to 'cuda' if available
device = 'cpu'

In [ ]:
from tqdm.notebook import tqdm as tqdm

In [ ]:
epochs = 10
lr = 1e-4

optimizer = torch.optim.Adam(model.parameters(), lr = lr)
loss_fn = torch.nn.BCEWithLogitsLoss()

model.train();

In [ ]:
losses = []

for epoch in tqdm(range(epochs)):
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()

        # forward pass
        y_hat = model(x)
        loss = loss_fn(y_hat, y)
        
        # step
        loss.backward()
        optimizer.step()
        
    losses.append(loss.item())